In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d,splrep,splev,PchipInterpolator

### Remove bad points, sort, to get M vs T curve

In [ ]:
filename = 'ppblsmocro03A-IP45deg-MvsT-cool7T-100Oewarm-20180717.rso.dat'

In [ ]:
MvT = np.genfromtxt(filename, delimiter=',', names=True, skip_header=30)

In [ ]:
offset = np.min(MvT['Long_Moment_emu'])
shifted = MvT['Long_Moment_emu'] - offset

In [ ]:
MvT.dtype.names

In [ ]:
%matplotlib notebook
plt.plot(MvT['Temperature_K'], MvT['Long_Reg_Fit'],'.-')
plt.xlabel('Temperature (K)')
plt.tight_layout()

In [ ]:
badfit = np.where(MvT['Long_Reg_Fit'][:210]<.95)[0]
temp = np.delete(MvT['Temperature_K'],badfit)
tsort = np.argsort(temp)
temp = temp[tsort]
moment = np.delete(shifted,badfit)[tsort]

In [ ]:
%matplotlib notebook
plt.plot(temp, moment*1e5,'.-')
plt.xlabel('Temperature (K)')
plt.ylabel(r'Moment ($10^{-5}$ emu)')
plt.tight_layout()

### Calculate Curie Temperature

In [ ]:
temps = np.linspace(310,360,(50*4)+1)

In [ ]:
#spline
mom = splrep(temp,moment)
firstderiv = splev(temps,mom,der=1)

In [ ]:
#Pchip
mom = PchipInterpolator(temp,moment)
firstderiv = (mom.derivative())(temps)

In [ ]:
%matplotlib notebook
plt.plot(temps, firstderiv,'.-')

In [ ]:
Tc = temps[np.argmin(firstderiv)]
print Tc

In [ ]:
Tcerr = (temps[np.argmin(firstderiv)+1]-temps[np.argmin(firstderiv)-1])/4.
print Tcerr

In [ ]:
#thickness = 
#area = etched area

#volume = area*thickness
#mag = moment/volume

In [ ]:
np.savez(filename.rpartition('.')[0], temperature = temp, moment = moment, TC = Tc, errTC = Tcerr#,
         #magnetizationemucc = mag
        )